## **Generation of Methanol Synthesis Dataset for Predictive Maintenance**

This dataset is synthetically generated for predictive maintenance in industrial methanol synthesis, specifically for a double-pass plug flow reactor (PFR) with 90% methanol removal after the first pass. Comprising 5000 samples, it is designed to closely mimic real-world industrial conditions using a kinetic model, so that it can be efficiently used to predict the Overall Yield after both passes of an industrial methanol synthesis reaction based on input conditions and identify ideal settings to achieve a maximum industrial yield of 70% or above. Low-yield predictions can trigger maintenance actions, such as change of reaction conditions or process optimization, to ensure consistent performance.

**Overall Reaction**: CO + $2H_2$ == $CH_3OH$

### Assumptions

To generate a dataset with entries that obeys the principles of chemical engineering and real-life industrial reactions, the following assumptions were made:

#### Reactor and Operating conditions
1. The reactor is an ideal __Plug Flow Reactor__ (PFR) i.e. there is no axial dispersion. It is also a double pass with 90% removal of product (methanol) after the first pass.
2. Operating conditions are at steady state, ideal gas behaviour,(PV=nRT) where;
   - Temperature (K): Ranges from 473–573 K, covering both suboptimal (low yield) and optimal (high yield) temperatures.
   - Pressure (bar): Ranges from 50–150 bar, aligning with industrial standards.
   - Gas constant (R): 8.314 J/mol.K
   - Residence times per pass is:
      - First pass residence time (s): Ranges from 5-20 s
      - Second pass residence time (s): Ranges from 10-30 s.
   Volume is fixed at 2000m³.
3. Feed composition is made up of pure CO and pure $H_2$ in molar ratio of 1:2.

#### Kinetics and Catalysis
4. Catalyst: Cu/ZnO/$Al_2O_3$.
5. Reaction Mechanism: Langmuir-Hinshelwood mechanism employed with the rate-limiting step being the slowest reaction on the catalyst surface, where adsorbed carbon monoxide, $CO*$ and hydrogen, $H*$ react to form a formyl intermediate, $HCO*$, releasing a free catalytic site $*$:   __CO* + H* ---> HCO* + *__ .
6. Kinetic Parameters: 
   - Pre-exponential Factor (A): 1*10¹⁰ mol/m³s
   - Activation Energy (Ea): 70000 J/mol
7. Adsorption constants for the different species
   - CO: $K_{CO}$ = 0.01 bar⁻¹,  △$H_{CO}$ = -20000 J/mol
   - $H_2$: $K_{H_2}$ = 0.005 bar⁻¹,   △$H_{H_2}$ = -15000 J/mol
   - $CH_3OH$: $K_{CH_3OH}$ = 0.1 bar⁻¹,  △$H_{CH_3OH}$ = -30000 J/mol
8. Equilibrium constant,$K_{eq}$ is given in terms of pressure i.e. $K_p$. $K_p$ is given as an empirical fit and not purely thermodynamic as it is widely accepted industrially for $K_p$ to be within the range of 0.01 bar⁻¹ and 0.03 bar⁻¹. Hence $K_p$ = 0.0147 bar⁻¹.
9. A conversion cap of 0.5 per pass is in place to reflect true industrial conditions of maximum conversion per pass.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.optimize import brentq

In [2]:
# Set random seed for reproducibility
np.random.seed(7)

# Constants
R = 8.314  # J/mol·K
A = 1e10  # mol/m³·s (realistic for industrial catalyst)
Ea = 70000  # J/mol
K_CO0, dH_CO = 1e-2, -20000       # bar^-1, J/mol
K_H20, dH_H2 = 5e-3, -15000       # bar^-1, J/mol
K_CH3OH0, dH_CH3OH = 1e-1, -30000 # bar^-1, J/mol
K_eq = 0.0147 # bar^-1
V = 2000 # m³
Conversion_cap = 0.5
Percentage_of_Product_passed = 0.9

# Reaction conditions ranges
n_samples = 5000 # number of samples
Temperature = np.random.uniform(473, 573, n_samples)         # Temperature (K)
Pressure = np.random.uniform(50, 150, n_samples)          # Pressure (bar)
res_time_1 = np.random.uniform(5, 20, n_samples)   # Residence time (s)
res_time_2 = np.random.uniform(10, 30, n_samples)   # Residence time (s)

In [3]:
# Generate initial dataset with reaction conditions ranges
def generate_initial_dataset():
    dataset = np.column_stack((Temperature, Pressure, res_time_1, res_time_2))
    columns = ["Temperature (K)", "Pressure (bar)", "Residence Time (s)_1", "Residence Time (s)_2"]
    df = pd.DataFrame(dataset, columns=columns)
    return df

df = generate_initial_dataset()    

df.head()

,Temperature (K),Pressure (bar),Residence Time (s)_1,Residence Time (s)_2
0,480.630829,91.621243,18.100214,13.236934
1,550.991879,68.001329,6.006925,24.561671
2,516.840923,70.676729,17.977576,19.551870
3,545.346518,95.531033,15.972852,16.805256
4,570.798951,54.649330,17.036762,22.303915


In [4]:
# Calculated Parameters
# Arrhenius Constant
def arrhenius_constant():
    K_Arrhenius = A*np.exp(-Ea/(R*Temperature))
    return K_Arrhenius

K_Arrhenius = arrhenius_constant()    


# Adsorption Constants
def adsorption_constants():
    K_CO = K_CO0*np.exp(-dH_CO/(R*Temperature))
    K_H2 = K_H20*np.exp(-dH_H2/(R*Temperature))
    K_CH3OH = K_CH3OH0*np.exp(-dH_CH3OH/(R*Temperature))
    return K_CO,K_H2,K_CH3OH

K_CO,K_H2,K_CH3OH = adsorption_constants()


# Partial pressures for first pass
def first_pass_partial_pressures():
    P_CO = (1/3)*Pressure
    P_H2 = (2/3)*Pressure
    P_CH3OH = (0/3)*Pressure
    return P_CO,P_H2,P_CH3OH

P_CO,P_H2,P_CH3OH = first_pass_partial_pressures()


# Rate of reaction for first pass
def first_pass_rate_of_reaction():
    numerator = (K_Arrhenius*K_CO*P_CO*((K_H2*P_H2)**0.5))
    denominator = (1+(K_CO*P_CO)+((K_H2*P_H2)**0.5)+(K_CH3OH*P_CH3OH))**2
    r = numerator/denominator
    return r

rate_of_reaction1 = first_pass_rate_of_reaction()


# Conversion (X) for first pass
def rate_of_reaction_conversion_1():
    # first find the initial concentration of fluid
    C_CO = (P_CO*1e5)/(R*Temperature)
    # Use these to find the rate of reaction conversion
    X_r = (rate_of_reaction1*res_time_1)/C_CO
    return X_r,C_CO # note returned concentration so it could be used to find initial conditions of flow

rate_of_reaction_conversion1,C_CO = rate_of_reaction_conversion_1()


# Feed rate for first pass, these feed rates would used to determine material balance of feed rates for second pass
def first_pass_feed_rates():
    # first find the volumetric flow rate
    vol_rate = V/res_time_1
    # use the initial concentration and volumetric flow rate to find the flow conditions
    Feed_rate_C0 = C_CO*vol_rate
    Feed_rate_H2 = 2*Feed_rate_C0
    return Feed_rate_C0,Feed_rate_H2

Feed_rate_C0,Feed_rate_H2 = first_pass_feed_rates()  


# Solve for equilibrium conversion using K_eq and the non-linear cubic equation "x = keq * P * (1 - x)^³"
# Redefines the equation to be the function "f(x) = keq * P * (1 - x)^³ - x"
def first_pass_equilibrium_conversion(P_H2,K_eq):
    # This function directly below returns the non-linear cubic equation as "keq * P * (1 - x)^³ - x"
    def function(x, P,K_eq):
        return K_eq * (P**2) * (1 - x)**3 - x
    # roots is a list to contain all roots of the equation between 0 and 1. (That is all roots that are real numbers) 
    roots = []
    for P in P_H2:
        # Use brentq to find the root between 0 and 1
        x = brentq(function, 0, 1, args=(P,K_eq))
        roots.append(x)
    return roots

In [5]:
# update the initial dataset with the first conversion
def apply_first_conversion(df):
    df["Equilibrium Conversion"] = first_pass_equilibrium_conversion(P_H2,K_eq) 
    df["Rate conversion"] = rate_of_reaction_conversion1 
    # After finding the equilibrium conversion and the rate of reaction conversion, using the conversion cap, find the minimum first conversion
    df["First Conversion"] = pd.DataFrame({"Cap": Conversion_cap,
                                            "Rate": df["Rate conversion"],
                                            "Equilibrium": df["Equilibrium Conversion"]}).min(axis=1)
    columns = ["Rate conversion","Equilibrium Conversion"]
    # Dropping unnecessary columns
    df = df.drop(columns,axis=1)  
    return df

df = apply_first_conversion(df)
df.head(20)  

,Temperature (K),Pressure (bar),Residence Time (s)_1,Residence Time (s)_2,First Conversion
0,480.630829,91.621243,18.100214,13.236934,0.381845
1,550.991879,68.001329,6.006925,24.561671,0.500000
2,516.840923,70.676729,17.977576,19.551870,0.500000
3,545.346518,95.531033,15.972852,16.805256,0.500000
4,570.798951,54.649330,17.036762,22.303915,0.500000
5,526.849587,50.915276,7.689944,15.740624,0.500000
6,523.112046,108.978748,18.616806,16.563335,0.500000
7,480.205113,86.716202,11.200205,11.226374,0.250310
8,499.843898,145.343874,18.784522,16.864358,0.467482
9,522.988250,146.015354,14.929243,21.747529,0.500000


In [6]:
# Second pass conditions
def second_pass_initial_feed_rates():
    # Feed rate after first pass
    Feed_rate_C0_2 = Feed_rate_C0*(1-df["First Conversion"])
    Feed_rate_H2_2 = Feed_rate_H2*(1-df["First Conversion"])
    Feed_rate_CH3OH_2 = Feed_rate_C0*df["First Conversion"]
    # But recall, only  90% of methanol was removed, therefore 100-90% of product enters the second pass
    Feed_rate_CH3OH_2 = (1-Percentage_of_Product_passed)*Feed_rate_CH3OH_2
    # Hence total feed rate
    Total_feed_rate_second_pass = Feed_rate_C0_2+Feed_rate_H2_2+Feed_rate_CH3OH_2

    # Finding volumetric flow rate so we can find the total pressure of the second pass 
    vol_rate_2 = V/res_time_2
    # Hence the pressure
    Pressure_2 = (Total_feed_rate_second_pass*R*df["Temperature (K)"])/(vol_rate_2*1e5)
    return Feed_rate_C0_2,Feed_rate_H2_2,Feed_rate_CH3OH_2,Total_feed_rate_second_pass,Pressure_2

Feed_rate_C0_2,Feed_rate_H2_2,Feed_rate_CH3OH_2,Total_feed_rate_second_pass,Pressure_2 = second_pass_initial_feed_rates() 


# Partial pressure is mole fraction * total pressure, firstly we find the mole fractions in second pass
# Mole fractions
def mole_fractions():
    y_CO = Feed_rate_C0_2/Total_feed_rate_second_pass
    y_H2 = Feed_rate_H2_2/Total_feed_rate_second_pass
    y_CH3OH = Feed_rate_CH3OH_2/Total_feed_rate_second_pass
    return y_CO,y_H2,y_CH3OH

y_CO,y_H2,y_CH3OH = mole_fractions()


# Now partial pressures 
def partial_pressures():
    P_CO_2 = y_CO*Pressure_2
    P_H2_2 = y_H2*Pressure_2
    P_CH3OH_2 = y_CH3OH*Pressure_2
    return P_CO_2,P_H2_2,P_CH3OH_2

P_CO_2,P_H2_2,P_CH3OH_2 = partial_pressures()


# Now we proceed to finding the rate of reaction of the second pass
def second_pass_rate_of_reaction():
    numerator2 = (K_Arrhenius*K_CO*P_CO_2*((K_H2*P_H2_2)**0.5))
    denominator2 = (1+(K_CO*P_CO_2)+((K_H2*P_H2_2)**0.5)+(K_CH3OH*P_CH3OH_2))**2
    r2 = numerator2/denominator2
    return r2

rate_of_reaction2 = second_pass_rate_of_reaction() 


# Finding the conversion (X) for the second pass
def rate_of_reaction_conversion_2():
    # To find rate equation Conversion, we must first find concentration of the second pass
    C_CO_2 = (P_CO_2*1e5)/(R*Temperature)
    X_r2 = (rate_of_reaction2*res_time_2)/C_CO_2
    X_r2
    return X_r2

rate_of_reaction_conversion2 = rate_of_reaction_conversion_2()


# Solve for equilibrium conversion using K_eq and the non-linear cubic equation "x = keq * P * (1 - x)^³"
# Redefines the equation to be the function "f(x) = keq * P * (1 - x)^³ - x"
def second_pass_equilibrium_conversion(P_H2_2,K_eq):
    # This function directly below returns the non-linear cubic equation as "keq * P * (1 - x)^³ - x"
    def function(x, P,K_eq):
        return K_eq * (P**2) * (1 - x)**3 - x
    # roots is a list to contain all roots of the equation between 0 and 1. (That is all roots that are real numbers) 
    roots = []
    for P in P_H2_2:
        # Use brentq to find the root between 0 and 1
        x = brentq(function, 0, 1, args=(P,K_eq))
        roots.append(x)
    return roots

In [7]:
# update the dataset with the second conversion
def apply_second_conversion(df):
    df["Equilibrium Conversion"] = second_pass_equilibrium_conversion(P_H2,K_eq) 
    df["Rate conversion"] = rate_of_reaction_conversion2 
    # After finding the equilibrium conversion and the rate of reaction conversion, using the conversion cap, find the minimum first conversion
    df["Second Conversion"] = pd.DataFrame({"Cap": Conversion_cap,
                                            "Rate": df["Rate conversion"],
                                            "Equilibrium": df["Equilibrium Conversion"]}).min(axis=1)
    columns = ["Rate conversion","Equilibrium Conversion"]
    # Dropping unnecessary columns
    df = df.drop(columns,axis=1) 
    return df                                       
df = apply_second_conversion(df)  

df.head(20)

,Temperature (K),Pressure (bar),Residence Time (s)_1,Residence Time (s)_2,First Conversion,Second Conversion
0,480.630829,91.621243,18.100214,13.236934,0.381845,0.014683
1,550.991879,68.001329,6.006925,24.561671,0.500000,0.053857
2,516.840923,70.676729,17.977576,19.551870,0.500000,0.063860
3,545.346518,95.531033,15.972852,16.805256,0.500000,0.131096
4,570.798951,54.649330,17.036762,22.303915,0.500000,0.500000
5,526.849587,50.915276,7.689944,15.740624,0.500000,0.051992
6,523.112046,108.978748,18.616806,16.563335,0.500000,0.051355
7,480.205113,86.716202,11.200205,11.226374,0.250310,0.017296
8,499.843898,145.343874,18.784522,16.864358,0.467482,0.012699
9,522.988250,146.015354,14.929243,21.747529,0.500000,0.020931


In [8]:
# Now find the overal yield
def apply_overall_yield(df):
    df["Overall Yield"] = df["First Conversion"]+((1-df["First Conversion"])*df["Second Conversion"])
    # Drop first and second conversion to avoid data leakage 
    conversions = ["First Conversion","Second Conversion"]
    df = df.drop(conversions,axis=1)
    # reindex dataframe
    df.index = np.arange(1,len(df)+1)
    return df

df = apply_overall_yield(df)
df.head(20)

,Temperature (K),Pressure (bar),Residence Time (s)_1,Residence Time (s)_2,Overall Yield
1,480.630829,91.621243,18.100214,13.236934,0.390921
2,550.991879,68.001329,6.006925,24.561671,0.526929
3,516.840923,70.676729,17.977576,19.551870,0.531930
4,545.346518,95.531033,15.972852,16.805256,0.565548
5,570.798951,54.649330,17.036762,22.303915,0.750000
6,526.849587,50.915276,7.689944,15.740624,0.525996
7,523.112046,108.978748,18.616806,16.563335,0.525677
8,480.205113,86.716202,11.200205,11.226374,0.263277
9,499.843898,145.343874,18.784522,16.864358,0.474244
10,522.988250,146.015354,14.929243,21.747529,0.510466


Download the synthetic generated methanol synthesis dataset used for building predictive model

In [9]:
df.to_csv('/Users/apple/Desktop/Methanol Synthesis Project/data/synthetic_methanol_synthesis_data.csv', index=False)